In [1]:
from unidecode import unidecode
from collections import defaultdict

In [2]:
def normalize(text):
    stripped = strip_non_alphanum(text)
    return unidecode(stripped.lower())

def strip_non_alphanum(text):
    return ''.join([ c for c in filter(lambda c: c == ' ' or c.isalnum(), text) ]).rstrip(' ')

In [29]:
def load_bigrams_norm(filename, k=10):
    bigrams = defaultdict(lambda: {}, {})
    normalized_bigrams = defaultdict(lambda: defaultdict(lambda: 0, {}), {})
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            n, word1, word2 = line.rstrip('\n').split(' ')
            n = int(n)

            if n < k: continue

            bigrams[word1][word2] = n

            norm_word1 = normalize(word1)
            word1_without_nonalnum = strip_non_alphanum(word1)
            norm_word2 = normalize(word2)
            word2_without_nonalnum = strip_non_alphanum(word2)

            normalized_bigrams[norm_word1][word1_without_nonalnum] += 1
            normalized_bigrams[norm_word2][word2_without_nonalnum] += 1
    return bigrams, normalized_bigrams     

In [30]:
bigrams, normalized_bigrams = load_bigrams_norm('./poleval_2grams.txt', 15)

In [46]:
def flatten(xss):
    return [x for xs in xss for x in xs]


def get_word_alts(word, normalized_bigrams):
    if word in normalized_bigrams:
        return normalized_bigrams[word]
    return { word: 1 }


def get_alts(words, normalized_bigrams):
    return list(map(lambda word: get_word_alts(word, normalized_bigrams), words))


def best_predecessor_score(word, predecessors, bigrams):
    best = (-1, None)
    for predecessor in predecessors:
        if predecessor in bigrams and word in bigrams[predecessor]:
            candidate = (bigrams[predecessor][word], predecessor)
            best = max(best, candidate)
    return best


def best_successor_score(word, successors, bigrams):
    best = (-1, None)
    if word in bigrams:
        for successor in successors:
            if successor in bigrams[word]:
                candidate = (bigrams[word][successor], successor)
                best = max(best, candidate)
    return best

In [62]:
from random import choice


def restore_diactrics(sentence, bigrams, normalized_bigrams):
    alts = get_alts(sentence.split(' '), normalized_bigrams)

    score = [{}]
    prev = [{}]
    for alt in alts[0]:
        score[0][alt] = 1
        prev[0][alt] = '<START>'
    
    for i in range(1, len(alts)):
        score.append({})
        prev.append({})

        for alt in alts[i]:
            s, word = best_predecessor_score(alt, alts[i-1], bigrams)
            if word is None:
                word = choice([ alt for alt in alts[i-1] ])
            score[-1][alt] = s
            prev[-1][alt] = word
    
    # TODO: use supertags

    val, last_word = max([ (c, w) for (w, c) in score[-1].items() ])
    result = [last_word]

    for i in range(len(alts)-1, 0, -1):
        result.append(prev[i][result[-1]])
    
    result.reverse()
    return [result[0].capitalize()] + result[1:]


In [63]:
# sentence = "Nie warto było"
sentence = "Uderzył go w żebra"
normalized_sentence = normalize(sentence)
restore_diactrics(normalized_sentence, bigrams, normalized_bigrams)

--- prevs ---
[{'uderzył': '<START>'}, {'go': 'uderzył', 'gó': 'uderzył'}, {'w': 'go'}, {'żebra': 'w', 'zebra': 'w'}]


['Uderzył', 'go', 'w', 'żebra']

In [53]:
sentence = "Zebra ma cztery nogi"
normalized_sentence = normalize(sentence)
restore_diactrics(normalized_sentence, bigrams, normalized_bigrams)

--- prevs ---
[{'żebra': '<START>', 'zebra': '<START>'}, {'ma': 'żebra', 'mą': 'zebra'}, {'cztery': 'ma'}, {'nogi': 'cztery'}]


['żebra', 'ma', 'cztery', 'nogi']

In [55]:
'ma' in bigrams['żebra']

False

In [57]:
print(bigrams['żebra'])

{'(': 15, ',': 127, 'i': 67, '.': 105}
